<a href="https://colab.research.google.com/github/hmg8906/CienciaDatosCN/blob/main/DefoRisk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# =============================================================================
# Script: tune_rf_deforestation_model
# Autor: Henry García-Diego
# Descripción:
# Este script entrena un modelo de riesgo de deforestación utilizando Google Earth Engine
# y Random Forest, evaluando distintas combinaciones de hiperparámetros para encontrar
# el mejor rendimiento (según AUC). El modelo se entrena con variables ambientales,
# de accesibilidad y vegetación, y produce un mapa de probabilidad de riesgo de deforestación.
#
# Funcionalidades clave:
# - Preprocesamiento y apilado de variables predictoras desde colecciones GEE.
# - Muestreo estratificado de puntos con pérdida y sin pérdida forestal.
# - Entrenamiento y validación cruzada con múltiples combinaciones de hiperparámetros.
# - Selección del mejor modelo según AUC (área bajo la curva ROC).
# - Exportación a Google Drive del mapa de riesgo (GeoTIFF) y métricas del modelo (CSV).
# - Visualización del mapa resultante usando geemap.
#
# Entradas:
# - aoi: ee.FeatureCollection o ee.Geometry delimitando la zona de análisis.
# - num_samples: número de muestras a recolectar para cada clase (por defecto: 2000).
# - export_resolution: resolución espacial en metros para exportación de resultados (por defecto: 1000).
#
# Salidas:
# - Un mapa interactivo con la capa de riesgo de deforestación.
# - Exportación automática a Google Drive de:
#     - Raster GeoTIFF de probabilidades de deforestación.
#     - CSV con métricas de evaluación del mejor modelo.
#
# Variables utilizadas:
# - Pérdida forestal (Hansen et al. 2023)
# - Cobertura boscosa año 2000
# - Elevación y pendiente (SRTM)
# - Distancia a vías (TIGER Roads)
# - Textura y carbono del suelo (OpenLandMap)
# - NDVI promedio anual 2023 (MODIS)
#
# Métricas evaluadas:
# - AUC (para seleccionar el mejor modelo)
# - Accuracy
# - Precision
# - Recall
# - F1-score
#
# Requiere:
# - geemap
# - sklearn
# - Google Earth Engine API autenticada
#
# Ejemplo de uso:
# >>> tune_rf_deforestation_model(mi_zona, num_samples=3000, export_resolution=500)
#
# ============================================================================

In [2]:
#inicializar y autenticar GEE
import ee
ee.Authenticate()
ee.Initialize(project="hmgarcia56ee1")

In [4]:
import geemap
import itertools
from sklearn.metrics import roc_auc_score, confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

def tune_rf_deforestation_model(aoi, num_samples=2000, export_resolution=1000):
    """
    Ajusta un modelo Random Forest para riesgo de deforestación usando GEE.
    Evalúa múltiples combinaciones de hiperparámetros y exporta los resultados.
    """
    try:
        forest    = ee.Image("UMD/hansen/global_forest_change_2023_v1_11")
        loss      = forest.select("loss")
        treecov   = forest.select("treecover2000")
        elev      = ee.Image("USGS/SRTMGL1_003")
        slope     = ee.Terrain.slope(elev)
        roads     = ee.FeatureCollection("TIGER/2016/Roads").map(lambda f: f.set("c", 1))
        road_rast = roads.reduceToImage(["c"], ee.Reducer.sum()).gt(0)
        road_dist = road_rast.fastDistanceTransform(30).rename("road_dist")
        soil_tex  = ee.Image("OpenLandMap/SOL/SOL_TEXTURE-CLASS_USDA-TT_M/v02").select("b0").rename("soil_tex")
        soil_carb = ee.Image("OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02").select("b0").rename("soil_carb")
        ndvi      = ee.ImageCollection("MODIS/006/MOD13Q1")\
                        .filterDate("2023-01-01", "2023-12-31")\
                        .select("NDVI").mean().multiply(0.0001).rename("ndvi")

        bands = [treecov, elev, slope, road_dist, soil_tex, soil_carb, ndvi]
        stack = ee.Image.cat(bands).float()

        loss_cand = stack.updateMask(loss.eq(1)).sample(region=aoi,
                            scale=30, numPixels=num_samples * 5,
                            seed=42, geometries=True)
        loss_pts = loss_cand.map(lambda f: f.set('loss', 1))
        n_loss   = loss_pts.size().getInfo()

        no_loss_cand = stack.updateMask(loss.eq(0)).sample(region=aoi,
                            scale=30, numPixels=n_loss * 5,
                            seed=84, geometries=True)
        no_loss_pts = no_loss_cand.map(lambda f: f.set('loss', 0)).limit(n_loss)
        pts = loss_pts.merge(no_loss_pts)

        fc    = pts.randomColumn('rand')
        train = fc.filter(ee.Filter.lt('rand', 0.7))
        test  = fc.filter(ee.Filter.gte('rand', 0.7))

        param_grid = {
            'numTrees': [50, 100, 150],
            'minLeafPopulation': [1, 5],
            'maxNodes': [None, 64]
        }

        all_combinations = list(itertools.product(
            param_grid['numTrees'],
            param_grid['minLeafPopulation'],
            param_grid['maxNodes']
        ))

        best_auc = 0
        best_params = None
        best_risk = None
        best_preds = None
        best_trues = None

        for trees, min_leaf, max_nodes in all_combinations:
            params = {
                'numberOfTrees': trees,
                'minLeafPopulation': min_leaf
            }
            if max_nodes:
                params['maxNodes'] = max_nodes

            clf = ee.Classifier.smileRandomForest(**params) \
                     .train(features=train,
                            classProperty='loss',
                            inputProperties=stack.bandNames()) \
                     .setOutputMode('PROBABILITY')

            risk = stack.classify(clf).rename('risk')
            eval_fc = risk.sampleRegions(
                collection=test,
                properties=['loss'],
                scale=export_resolution,
                tileScale=4
            )

            preds = eval_fc.aggregate_array('risk').getInfo()
            trues = eval_fc.aggregate_array('loss').getInfo()

            if len(preds) == len(trues) and len(preds) > 0:
                auc = roc_auc_score(trues, preds)
                if auc > best_auc:
                    best_auc = auc
                    best_params = (trees, min_leaf, max_nodes)
                    best_risk = risk
                    best_preds = preds
                    best_trues = trues

        if best_risk:
            print(f"\n🏆 Mejor combinación: Árboles={best_params[0]}, minLeaf={best_params[1]}, maxNodes={best_params[2]}")
            print(f"AUC óptimo: {best_auc:.4f}")

            # Exportar imagen de riesgo
            task = ee.batch.Export.image.toDrive(
                image=best_risk,
                description='best_def_risk',
                folder='GEE_Exports',
                fileNamePrefix='best_risk_probability_map',
                scale=export_resolution,
                region=aoi.geometry().bounds(),
                maxPixels=1e13,
                fileFormat='GeoTIFF'
            )
            task.start()
            print("🗺️ Exportación del mapa iniciada.")

            # Calcular métricas y exportarlas como CSV
            y_true = best_trues
            y_pred = [1 if p > 0.5 else 0 for p in best_preds]

            accuracy  = accuracy_score(y_true, y_pred)
            precision = precision_score(y_true, y_pred)
            recall    = recall_score(y_true, y_pred)
            f1        = f1_score(y_true, y_pred)

            print(f"\n📊 Métricas del mejor modelo:\n"
                  f"Accuracy:  {accuracy:.3f}\n"
                  f"Precision: {precision:.3f}\n"
                  f"Recall:    {recall:.3f}\n"
                  f"F1-score:  {f1:.3f}")

            metrics_fc = ee.FeatureCollection([
                ee.Feature(None, {
                    'Accuracy':  accuracy,
                    'Precision': precision,
                    'Recall':    recall,
                    'F1_score':  f1
                })
            ])

            metrics_task = ee.batch.Export.table.toDrive(
                collection=metrics_fc,
                description='metrics_csv',
                folder='GEE_Exports',
                fileNamePrefix='best_rf_metrics',
                fileFormat='CSV'
            )
            metrics_task.start()
            print("📁 Exportación de métricas iniciada (CSV → Drive carpeta GEE_Exports).")

            m = geemap.Map()
            m.centerObject(aoi, 6)
            m.addLayer(best_risk, {'min': 0, 'max': 1, 'palette': ['green','yellow','red']},
                       "Mejor Riesgo (probabilidad)")
            return m

        else:
            print("⚠️ No se encontró un modelo válido con los parámetros dados.")

    except Exception as e:
        print("❌ Error:", e)


In [5]:
aoi = ee.FeatureCollection("FAO/GAUL/2015/level1") \
         .filter(ee.Filter.eq("ADM1_NAME", "Amazonas"))

Map = tune_rf_deforestation_model(aoi, num_samples=4000, export_resolution=1000)
if Map:
    Map



🏆 Mejor combinación: Árboles=150, minLeaf=5, maxNodes=None
AUC óptimo: 0.7808
🗺️ Exportación del mapa iniciada.

📊 Métricas del mejor modelo:
Accuracy:  0.754
Precision: 0.748
Recall:    0.722
F1-score:  0.735
📁 Exportación de métricas iniciada (CSV → Drive carpeta GEE_Exports).
